# Label Type Detection
Train a model that can predict the label captured in a representitive image: phase, nuclear, fluorescent cytoplasm.

In [1]:
import os
import errno

import numpy as np

import deepcell

In [2]:
# Set up some global constants and shared filepaths

SEED = 213  # random seed for splitting data into train/test

ROOT_DIR = '/data'  # TODO: Change this! Usually a mounted volume
DATA_DIR = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models'))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs'))

MODEL_NAME = 'LabelDetectionModel'

MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME + '.h5')

# create directories if they do not exist
for d in (MODEL_DIR, LOG_DIR):
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

## Load the training data

Download data for nuclear, brightfield and fluorescent cytoplasm from `deepcell.datasets` and combine the data into a single training dataset.

The labels for each type of data are as follows:

- Nuclear = 0
- Phase = 1
- Fluorescent Cytoplasm = 2

In [3]:
# First, download the data from deepcell.datasets
import deepcell.datasets

# nuclear data (label type 0)
(X_train, y_train), (X_test, y_test) = deepcell.datasets.hela_s3.load_data(seed=SEED)
nuclear_train = {'X': X_train, 'y': y_train}
nuclear_test = {'X': X_test, 'y': y_test}

In [4]:
# brightfield phase data (label type 1)
(X_train, y_train), (X_test, y_test) = deepcell.datasets.phase.HeLa_S3.load_data(seed=SEED)
brightfield_train = {'X': X_train, 'y': y_train}
brightfield_test = {'X': X_test, 'y': y_test}

In [5]:
# flourescent cytoplasm data (label type 2)
(X_train, y_train), (X_test, y_test) = deepcell.datasets.cytoplasm.hela_s3.load_data(seed=SEED)
flourescent_train = {'X': X_train, 'y': y_train}
flourescent_test = {'X': X_test, 'y': y_test}

In [6]:
# Reshape each dataset to conform to the minimum size of 216
from deepcell.utils.data_utils import reshape_matrix

RESHAPE_SIZE = 216

all_train = [nuclear_train, brightfield_train, flourescent_train]
all_test = [nuclear_test, brightfield_test, flourescent_test]

for train, test in zip(all_train, all_test):
    train['X'], train['y'] = reshape_matrix(train['X'], train['y'], RESHAPE_SIZE)
    test['X'], test['y'] = reshape_matrix(test['X'], test['y'], RESHAPE_SIZE)

Reshaped feature data from (5760, 216, 256, 1) to (5760, 216, 216, 1)
Reshaped training data from (5760, 216, 256, 1) to (5760, 216, 216, 1)
Reshaped feature data from (1440, 216, 256, 1) to (1440, 216, 216, 1)
Reshaped training data from (1440, 216, 256, 1) to (1440, 216, 216, 1)
Reshaped feature data from (1651, 512, 512, 1) to (14859, 216, 216, 1)
Reshaped training data from (1651, 512, 512, 1) to (14859, 216, 216, 1)
Reshaped feature data from (413, 512, 512, 1) to (3717, 216, 216, 1)
Reshaped training data from (413, 512, 512, 1) to (3717, 216, 216, 1)
Reshaped feature data from (1651, 512, 512, 1) to (14859, 216, 216, 1)
Reshaped training data from (1651, 512, 512, 1) to (14859, 216, 216, 1)
Reshaped feature data from (413, 512, 512, 1) to (3717, 216, 216, 1)
Reshaped training data from (413, 512, 512, 1) to (3717, 216, 216, 1)


In [7]:
# Stack up our data as train and test

def make_y(batch, i):
    y = np.zeros((batch, 3))
    y[:, i] = 1
    return y

X_train = np.vstack([
    nuclear_train['X'],
    brightfield_train['X'],
    flourescent_train['X']
])

y_train = np.vstack([
    make_y(nuclear_train['y'].shape[0], 0),
    make_y(brightfield_train['y'].shape[0], 1),
    make_y(flourescent_train['y'].shape[0], 2)
])

X_test = np.vstack([
    nuclear_test['X'],
    brightfield_test['X'],
    flourescent_test['X']
])

y_test = np.vstack([
    make_y(nuclear_test['y'].shape[0], 0),
    make_y(brightfield_test['y'].shape[0], 1),
    make_y(flourescent_test['y'].shape[0], 2)
])

## Create the Label Detection Model

We are using the LabelDetectionModel from `deepcell.applications`

In [8]:
from deepcell.applications import LabelDetectionModel

# set use_pretrained_weights=False to start training from scratch
model = LabelDetectionModel(input_shape=X_train.shape[1:], use_pretrained_weights=False)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 216, 216, 1)       0         
_________________________________________________________________
image_normalization2d (Image (None, 216, 216, 1)       3721      
_________________________________________________________________
tensor_product (TensorProduc (None, 216, 216, 3)       6         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 216, 216, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 216, 216, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 108, 108, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 108, 108, 128)     73856     
__________

In [9]:
from tensorflow.keras.optimizers import SGD, Adam
from deepcell.utils.train_utils import rate_scheduler
from deepcell import losses


def loss_function(y_true, y_pred):
    return losses.weighted_categorical_crossentropy(
            y_true, y_pred,
            n_classes=len(np.unique(y_train)))


n_epoch = 20  # Number of training epochs
lr = 1e-3
optimizer = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
lr_sched = rate_scheduler(lr=lr, decay=0.9)
batch_size = 32

model.compile(optimizer, loss=loss_function, metrics=['accuracy'])

## Train model

In [10]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

# Initialize data generators for training
generator = ImageDataGenerator(
    rotation_range=180,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=(0.5, 2))

val_generator = ImageDataGenerator()  # No augmentation for validation data

In [11]:
from tensorflow.python.keras import callbacks

model.fit_generator(
    generator.flow(X_train, y_train, batch_size=batch_size), 
    steps_per_epoch=X_train.shape[0] // batch_size,
    epochs=n_epoch,
    validation_data=val_generator.flow(X_test, y_test, batch_size=batch_size),
    validation_steps=X_test.shape[0] // batch_size,
    callbacks=[
        callbacks.LearningRateScheduler(lr_sched),
        callbacks.ModelCheckpoint(
            MODEL_PATH,
            monitor='val_loss',
            verbose=1,
            save_best_only=True),
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10
        )
    ]
)

Epoch 1/20
1107/1108 [============================>.] - ETA: 0s - loss: 0.8642 - acc: 0.7551
Epoch 00001: val_loss improved from inf to 0.40321, saving model to /data/models/LabelDetectionModel.h5
1108/1108 [==============================] - 185s 167ms/step - loss: 0.8639 - acc: 0.7552 - val_loss: 0.4032 - val_acc: 0.8833
Epoch 2/20
1107/1108 [============================>.] - ETA: 0s - loss: 0.3728 - acc: 0.8967
Epoch 00002: val_loss improved from 0.40321 to 0.22479, saving model to /data/models/LabelDetectionModel.h5
1108/1108 [==============================] - 177s 160ms/step - loss: 0.3728 - acc: 0.8968 - val_loss: 0.2248 - val_acc: 0.9457
Epoch 3/20
1107/1108 [============================>.] - ETA: 0s - loss: 0.2345 - acc: 0.9414
Epoch 00003: val_loss did not improve from 0.22479
1108/1108 [==============================] - 177s 160ms/step - loss: 0.2346 - acc: 0.9414 - val_loss: 0.2909 - val_acc: 0.9351
Epoch 4/20
1107/1108 [============================>.] - ETA: 0s - loss: 0.175

## Test model predictions

In [12]:
from sklearn.metrics import confusion_matrix

Y = []
Y_pred = []

val_data = val_generator.flow(X_test, y_test, batch_size=1)

for i in range(1000):
    if i % 100 == 0:
        print(".", end="")

    lst, y_true = val_data.next()
    y_true = np.argmax(y_true, axis=-1)
    y_pred = np.argmax(model.predict(lst), axis=-1)
    Y.append(y_true)
    Y_pred.append(y_pred)
    
Y = np.concatenate(Y, axis=0)
Y_pred = np.concatenate(Y_pred, axis=0)

print("")
cm = confusion_matrix(Y, Y_pred)
print(cm)

..........
[[139   0  18]
 [  6 388  20]
 [  0   0 429]]
